In [1]:
import json
from urllib.request import urlopen
import pandas as pd
from pulp import *
import re

In [2]:
#Set number of available players
formations = [
    {'GKP': 1, 'DEF': 3, 'MID': 4, 'FWD': 3},
    {'GKP': 1, 'DEF': 3, 'MID': 5, 'FWD': 2},
    {'GKP': 1, 'DEF': 4, 'MID': 5, 'FWD': 1},
    {'GKP': 1, 'DEF': 4, 'MID': 4, 'FWD': 2},
    {'GKP': 1, 'DEF': 4, 'MID': 3, 'FWD': 3} 
]

#Set Budget

BUDGET = 90

In [3]:
class Dataset():
    
    def get_url(self, key, Id=None):
        base_url = 'https://fantasy.premierleague.com/drf'
        api = {
            'All': '/bootstrap-static',
            'Player': '/element-summary/{}',
            'Team':'/my-team/48049/{}',
            '1':'/bootstrap',
            '2':'/bootstrap-static',
            '3':'/bootstrap-dynamic',
            '4':'/events',
            'player_stats':'/elements',
            'player_positions':'/element-types',
            '7':'/fixtures',
            'teams':'/teams',
            '9':'/region',
            '10':'/transfers',
            '11':'/entry/{}',
            '12':'/entries',
            '13':'/my-team/{}',
            '14':'/leagues-entered/{}',
            '15':'/leagues-classic/{}',
            '16':'/leagues-classic-standings/{}',
            '17':'/leagues-h2h/{}',
            '18':'/leagues-h2h-standings/{}'
        }

        if Id is None:
            url = ''.join([base_url, api[key]])
        else:
            api_extension = api[key]
            api_extension = api_extension.format(Id)
            url = ''.join([base_url, api_extension])
        return url

    def pull_data(self, key, Id=None):
        json_data = json.load(urlopen(self.get_url(key, Id)))
        df = pd.read_json(json.dumps(json_data))
        return df

    def get_player_data(self, active=False):
        player_stats = self.pull_data('player_stats')
        player_positions = self.pull_data('player_positions')[['id', 'singular_name_short']]
        player_teams = self.pull_data('teams')[['code', 'name']]
        
        player_table = player_stats.merge(player_positions, left_on='element_type', right_on='id')
        player_table = player_table.merge(player_teams, left_on='team_code', right_on='code')
        player_table['now_cost'] = player_table['now_cost']/10
        if active:
            player_table = player_table.loc[player_table['chance_of_playing_next_round'] == 100]
            player_table = player_table.reset_index()
        return player_table
    
    def model_data(self):
        active_players = self.get_player_data(True)
        player_pool = active_players[['web_name', 'singular_name_short', 'now_cost', 'total_points', 'name']]
        player_pool.columns = ['web_name', 'position', 'cost', 'total_points', 'team_name']
        
        costs = {}
        points = {}
        
        for pos in player_pool.position.unique():
            pos_players = player_pool.loc[player_pool.position == pos]
            cost = list(pos_players[['web_name', 'cost']].set_index('web_name').to_dict().values())[0]
            point = list(pos_players[['web_name', 'total_points']].set_index('web_name').to_dict().values())[0]
            costs[pos] = cost
            points[pos] = point

        return (costs, points)
        

In [4]:
class TeamSolutions():
    def __init__(self, data, formations, budgets):
        self.formations = formations
        self.budgets = budgets
        self.costs, self.points = data
        self._vars = _vars = {pos: LpVariable.dict(pos, players, cat="Binary") for pos, players in self.points.items()}
        
    def solve_problem(self, formation, budget):
        #Starting the problem

        prob = LpProblem('Naive_total_score_max', LpMaximize)
        rewards = []
        Costs = []
        position_constraints = []

        #Reward

        for pos, players in self._vars.items():
            Costs += lpSum([self.costs[pos][player] * self._vars[pos][player] for player in players])
            rewards += lpSum([self.points[pos][player] * self._vars[pos][player] for player in players])
            prob += lpSum([self._vars[pos][player] for player in players]) <= formation[pos]

        prob += lpSum(rewards)
        prob += lpSum(Costs) <= budget
        
        prob.solve()
        return prob
   
    def print_summary(self, prob):
        div = '----------------------------\n'
        print("Variables:\n")
        score = str(prob.objective)
        constraints = [str(const) for const in prob.constraints.values()]
        for v in prob.variables():
            score = score.replace(v.name, str(v.varValue))
            constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
            if v.varValue != 0:
                print(v.name, "=", v.varValue)
        print(div)
        print("Constraints:")
        for constraint in constraints:
            constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
            if constraint_pretty != "":
                print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
        print(div)
        print("Score:")
        score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
        print("{} = {}".format(score_pretty, eval(score)))
    
    def summarise(self, prob):
        Players = []
        Budget = {}
        Score = {}
        
        score = str(prob.objective)
        constraints = [str(const) for const in prob.constraints.values()]
        
        for v in prob.variables():
            score = score.replace(v.name, str(v.varValue))
            constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
            if v.varValue != 0:
                Players.append(v.name)


        for player in Players:
            position = player[0:3]
            web_name = player[4:].replace("_"," ")
            Budget[web_name] = self.costs[position][web_name]
            Score[player] = self.points[position][web_name]

        Budget['Total'] = sum(Budget.values())
        Score['Total'] = sum(Score.values())
        return {'Players': Players,'Budget': Budget,'Score': Score}

    def solve_for_all_formations(self):
        possible_teams = []
        for formation in formations:
            possible_team = self.solve_problem(formation, self.budgets)
            team_summary = self.summarise(possible_team)
            possible_teams.append(team_summary)
        return possible_teams
            
            

In [5]:
dataset = Dataset()
data = dataset.model_data()

In [6]:
team_model = TeamSolutions(data, formations, BUDGET)

In [7]:
possible_teams = team_model.solve_for_all_formations()

In [8]:
dataset.get_player_data()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code_x,cost_change_event,cost_change_event_fall,cost_change_start,...,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,id_y,singular_name_short,code_y,name
0,0,3,130,100.0,100.0,1,11334,0,0,-1,...,112720,814,0.0,4.9,Cech,0,1,GKP,3,Arsenal
1,0,1,105,NaN,NaN,0,80201,0,0,-2,...,82552,886,0.7,3.8,Leno,0,1,GKP,3,Arsenal
2,0,0,0,NaN,NaN,0,98980,0,0,0,...,3495,284,0.0,0.0,Martínez,0,1,GKP,3,Arsenal
3,0,0,0,0.0,0.0,0,51507,0,0,-1,...,4245,33,0.0,0.0,Koscielny,0,2,DEF,3,Arsenal
4,4,3,222,100.0,100.0,2,98745,0,0,-1,...,579994,9283,0.3,7.6,Bellerín,2,2,DEF,3,Arsenal
5,1,2,124,0.0,0.0,2,38411,0,0,0,...,166005,3134,0.0,5.5,Monreal,1,2,DEF,3,Arsenal
6,0,5,129,NaN,NaN,1,156074,0,0,-1,...,26158,636,0.6,5.0,Holding,1,2,DEF,3,Arsenal
7,0,1,221,NaN,NaN,2,69140,0,0,-1,...,56004,1037,0.4,6.5,Mustafi,2,2,DEF,3,Arsenal
8,2,3,57,100.0,100.0,0,111457,0,0,-1,...,22533,286,1.0,3.1,Kolasinac,0,2,DEF,3,Arsenal
9,0,0,0,0.0,0.0,0,233963,0,0,-3,...,8944,29,0.0,0.0,Mavropanos,0,2,DEF,3,Arsenal
